# Тест Математик

Цокт**о** Жигмытов, телеграм @tsokto

## Задание 1

В урне лежат черные и белые шары. Какова вероятность, что среди вытянутых шаров будет конкретное количество белых?  
* Белых шаров: 12. 
* Черных шаров: 18.
* Достаётся всего шаров: 10.
* Среди них белых должно быть: 3. 

Ответ: 0.2330263439708717

- тж результат приведён в Excel-таблице
- ниже приведен код расчёта + проверка исходов на полноту

In [ ]:
from math import factorial as fct

In [ ]:
def shary(W=12, B=18, n=10, w=3):
    CwW = fct(W) / (fct(W-w) * fct(w))
    Cn_wB = fct(B) / (fct(B-n+w) * fct(n-w))
    CnWB = fct(W + B) / (fct(W + B - n) * fct(n))
    return CwW * Cn_wB / CnWB  


In [ ]:
shary()

0.2330263439708717

In [ ]:
# проверка исходов  на полноту

p_list=[]

for i in range(11):
    p_list.append(shary(w=i))
print(p_list)
print()
print(f'сумма: {sum(p_list)}') # должно быть =1 

[0.0014564146498179482, 0.01941886199757264, 0.09612336688798458, 0.2330263439708717, 0.3058470764617691, 0.2258563026179218, 0.09410679275746742, 0.021510124058849694, 0.002520717663146449, 0.00013180223075275548, 2.1967038458792583e-06]

сумма: 0.9999999999999998


## Задание 2

Дано описание игры all ways slot (таблица Excel).

Необходимо рассчитать для базовой игры:
1.  Вероятность выпадения каждой комбинации;
2.  Средний выигрыш в монетах;
3.  Средний процент возврата средств
4.  Частоту выпадения бонусных спинов (Free Spins)
5.  Среднее количество бонусных спинов, приходящееся на 1 спин базовой игры 

Ответ:

1.	Вероятности выпадения каждой комбинации: см. файл Excel
2.	Средний выигрыш в монетах: ~ 947.5
3.	Средний процент возврата средств: ~94.75%
4.	Частота выпадения бонусных спинов (общая): 0.125015884
5.	Среднее кол-во бонусных спинов на 1 спин базовой игры: 0.111123661

Подробнее в таблице Excel

## Задание 3

Написать симулятор вышеописанной игры all ways slot.

1. ~~Движок: игровое поле, подсчёт комбинаций, подсчёт суммы~~
2. ~~Базовый симулятор: проверка запуском ячейки~~
3. ~~График: RTP, выигрыши, средний выигрыш~~
4.  Продвинутый движок: бонусные вращения, seed values via random.org API
5. Продвинутый симулятор: ставки (?), баланс, интерактив, псевдо(?)графика
6. Продвинутый график: точка безубыточности (малой убыточности?), минимальная достаточно точная выборка
7. Анализ: отклонение, волатильность и т.д. 

### Код 

In [1]:
from random import choice

import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [12]:
class SlotData():
    '''ver. 0.1
    Содержит данные о машине: барабаны, таблица платежей, 
    TODO: сделать обработку размера ставки, баланса, посева ГПСЧ'''


    # барабаны
    reels = {1: ('10', 'Sc', '04', '05', '08', '11', '04', '05', '09', '08', '07', '11', '10', '06', '07', '09', '08', '10', '03', '09', '11', '06', '09',),
             2: ('06', 'Sc', '03', '08', '07', '06', '09', '11', 'Wi', '08', '04', '05', '10', '11', '04', '10', '08', '07', '09', '11', '05', '10', '09',),
             3: ('06', 'Sc', '03', '08', '07', '06', '09', '11', 'Wi', '08', '04', '05', '10', '11', '04', '10', '08', '07', '09', '11', '05', '10', '09',),
             4: ('06', 'Sc', '03', '08', '07', '06', '09', '11', 'Wi', '08', '04', '05', '10', '11', '04', '10', '08', '07', '09', '11', '05', '10', '09',),
             5: ('06', 'Sc', '03', '08', '07', '06', '09', '11', 'Wi', '08', '04', '05', '10', '11', '04', '10', '08', '07', '09', '11', '05', '10', '09',),
             }
    
    # таблица выплат
    paytable = {('Sc',5): 0,     ('Sc',4): 0,     ('Sc',3): 0,
                ('03',5): 20000, ('03',4): 10000, ('03',3): 2000,
                ('04',5): 10000, ('04',4): 5000,  ('04',3): 1000,
                ('05',5): 6000,  ('05',4): 4000,  ('05',3): 1000,
                ('06',5): 5000,  ('06',4): 2000,  ('06',3): 1000,
                ('07',5): 4000,  ('07',4): 1000,  ('07',3): 500,
                ('08',5): 4000,  ('08',4): 1000,  ('08',3): 500,
                ('09',5): 2000,  ('09',4): 500,   ('09',3): 100,
                ('10',5): 2000,  ('10',4): 500,   ('10',3): 100,
                ('11',5): 2000,  ('11',4): 500,   ('11',3): 100,
                }
    
    # таблица scatters → freespins
    freespins_table = {5:10, 4:8, 3:6, 2:0, 1:0, 0:0}

    # ставка = 1000
    # баланс = 0
    
    def __init__(self, bet=1000, balance=0, seed=42):
        self.bet = bet
        self.balance = balance        
        if type(seed) in {int, tuple, float, str}:
            self.seed = seed
        else:
            self.seed = str(seed)


In [14]:
class PlayScreen():
    '''Формирует игровое поле, считает результат'''

    # состоит из трёх линий: top, middle, bottom    
    top_line, mid_line, btm_line, hits = [], [], [], []
    freespins = 0
    scatters = 0
    win = 0

    def __init__(self, slot, to_seed=False):
        self.reels = slot.reels
        self.slot = slot
        if to_seed:
            self.seed = slot.seed

    
    def pull_handle(self):
        self.top_line, self.mid_line, self.btm_line, self.hits = [], [], [], []
        for i in self.reels.values():
            j_mid = choice(range(23))    
            j_top = (j_mid+1)%23
            j_btm = (j_mid-1)%23            
            self.top_line.append(i[j_top])
            self.mid_line.append(i[j_mid])
            self.btm_line.append(i[j_btm])
    
        # проверка наличия scatter + подсчёт бонусных спинов
        self.scatters = self.top_line.count('Sc') + self.mid_line.count('Sc') + self.btm_line.count('Sc')
        self.freespins = self.slot.freespins_table[self.scatters]

        # линии для тестирования: раскомментировать и вписать нужные значения     
        
        # self.top_line = ['04', '04', '04', '04', '04',]
        # self.mid_line = ['05', '08', '08', '08', '08',]
        # self.btm_line = ['09', 'Wi', 'Wi', 'Wi', 'Wi',]

        # self.top_line = ['10', 'Wi', '08', '10', '03',]
        # self.mid_line = ['11', '11', '03', '05', 'Sc',]
        # self.btm_line = ['07', '09', 'Sc', '11', '06',]

        # подсчёт значений 
        for el in [self.top_line[0], self.mid_line[0], self.btm_line[0]]:
            count = 1
            trigger = True
            i = 1

            # сразу проверяем третью колонку
            # если там пусто, то комбинаций нет   
            if el not in [self.top_line[2], self.mid_line[2], self.btm_line[2]] and 'Wi' not in [self.top_line[2], self.mid_line[2], self.btm_line[2]]:
                trigger = False
            
            # если третья колонка не пустая, считаем 
            while i <= 4 and trigger:
                if el not in [self.top_line[i], self.mid_line[i], self.btm_line[i]] and 'Wi' not in [self.top_line[i], self.mid_line[i], self.btm_line[i]]:
                    trigger = False
                else:
                    column = [self.top_line[i], self.mid_line[i], self.btm_line[i]]
                    column[:] = [el if x == 'Wi' else x for x in column]
                    count += column.count(el)
                i += 1
            
            if count >=3:        
                self.hits.append(tuple([el, count]))
        
        # подсчитываем сумму
        # если сочетаний насчиталось больше 5, значит сыграл Wild
        # каждый Wild в этом случае удваивает выигрыш

        self.win = sum([self.slot.paytable[(hit[0],5)] * 2 ** (hit[1] - 5) if hit[1] > 5 else self.slot.paytable[hit] for hit in self.hits])    

    def show(self):
        # вывод игрового поля
        print('-'*27)
        print('-= ALL WAYS SLOT MACHINE =-')
        print('-'*27)
        print(' | ', end='')
        for e in self.top_line:
            print(e, end=' | ') 
        print('\n')
        print(' | ', end='') 
        for e in self.mid_line:
            print(e, end=' | ') 
        print('\n') 
        print(' | ', end='') 
        for e in self.btm_line:
            print(e, end=' | ')
        print('\n') 
        # print(f'SCATTERS: {self.scatters}')
        print(f'BONUS SPINS: {self.freespins}')
        print(f'HITS: {self.hits}')
        print(f'YOUR WIN: {self.win}')


In [15]:
def simulate(base_spins=10000):
    '''Функция для симуляции большого количества игр + визуализация. По умолчанию попыток 10000'''

    total_pulls = 0
    sum_wins = 0
    profit = []
    wins = []
    rtp = []
    # eleven_three = 0

    i = 0
    spins = base_spins

    while i <= spins:
        my_screen.pull_handle()
        spins += my_screen.freespins
        total_pulls +=1

        # список выигрышей за каждое вращение
        wins.append(my_screen.win)

        # сумма выигрышей нарастающим итогом
        sum_wins += my_screen.win

        # средний выигрыш
        profit.append(sum_wins / total_pulls)

        # средний возврат средств 
        rtp.append(sum_wins * 100 / (total_pulls * 1000))

        i += 1

    print (f'Итоговое количество вращений: {total_pulls}')
    print (f'Их них базовых вращений {base_spins}, бонусных {total_pulls-base_spins}')
    print (f'Вероятность бонусных вращений: {(total_pulls-base_spins)/base_spins}')

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=[i for i in range(total_pulls)], y=wins, name='Wins'),
                  secondary_y=False,
                  )
    fig.add_trace(go.Scatter(x=[i for i in range(total_pulls)], y=profit, name='Average win'),
                  secondary_y=False,
                  )
    fig.add_trace(go.Scatter(x=[i for i in range(total_pulls)], y=rtp, name='RTP, %'),
                  secondary_y=True,
                  )
    fig.update_layout(title="<b>Wins, Average Win, RTP%</b>")
    fig.update_xaxes(title_text=f"Количество попыток: {total_pulls}")

    fig.update_yaxes(title_text="<b>Выигрыш в монетах</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Return to Player, %</b>", secondary_y=True)

    fig.show()

### Check

In [16]:
# инициализируем машину и игровой экран

my_slot = SlotData()
my_screen = PlayScreen(my_slot)

In [17]:
# Запустите ноутбук для попытки

my_screen.pull_handle()
my_screen.show()

---------------------------
-= ALL WAYS SLOT MACHINE =-
---------------------------
 | 11 | 10 | 11 | 05 | 03 | 

 | 07 | 05 | 09 | 11 | Sc | 

 | 08 | 11 | 06 | 09 | 06 | 

BONUS SPINS: 0
HITS: [('11', 4)]
YOUR WIN: 500


### Simulate

In [18]:
# аргумент - количество попыток, по умолчанию 10000

simulate()

Итоговое количество вращений: 11085
Их них базовых вращений 10000, бонусных 1085
Вероятность бонусных вращений: 0.1085
